In [1]:
import json
import geopandas as gpd
import pandas as pd

# Constants
PATH = '../data/location/grid.gpkg'

def combine_pois() -> gpd.GeoDataFrame:
    poi_mof = gpd.read_file('../data/demand/poi_mof.geojson', columns=['Name', 'Category']).to_crs(epsg=3826)
    poi_osm = gpd.read_file('../data/demand/poi_osm.geojson', columns=['Name', 'Category']).to_crs(epsg=3826)
    poi_mof['Source'] = pd.Series(['MOF' for i in range(len(poi_mof))])
    poi_osm['Source'] = pd.Series(['OSM' for i in range(len(poi_osm))])
    return gpd.GeoDataFrame(pd.concat([poi_mof, poi_osm]).reset_index(drop=True)[['Name', 'Category', 'Source', 'geometry']])

pois = combine_pois()
pois.to_file(PATH, driver='GPKG', layer='poi')
display(pois.head(10))
print(f'{len(pois)} rows x {len(pois.columns)} cols')

,Name,Category,Source,geometry
0,一江鎖印店,零售,MOF,POINT (303619.526 2771844.873)
1,得商有限公司,批發,MOF,POINT (304602.2 2773024.951)
2,滴二石文創有限公司,餐飲,MOF,POINT (304623.681 2773024.845)
3,碧達拉有限公司,批發,MOF,POINT (304623.681 2773024.845)
4,大馨花藝工作室,零售,MOF,POINT (304637.096 2773019.919)
5,嘉橙有限公司,批發,MOF,POINT (304867.453 2772823.414)
6,萬安國際股份有限公司,批發,MOF,POINT (304867.453 2772823.414)
7,洪裕國際股份有限公司,批發,MOF,POINT (304867.453 2772823.414)
8,旺普電通股份有限公司,批發,MOF,POINT (304867.453 2772823.414)
9,旺普網路資訊股份有限公司,批發,MOF,POINT (304867.453 2772823.414)


141415 rows x 4 cols


In [ ]:
# Imports
grid = gpd.read_file(PATH, layer='original-grid')
taipei = gpd.read_file(PATH, layer='taipei')

# Save to files
filtered_grid = grid[grid.intersects(taipei.iloc[0, 0])]
filtered_grid.to_file(PATH, driver='GPKG', layer='filtered-grid')

del filtered_grid, grid, taipei

In [2]:
# Calculate the demand multiplier for each category in each grid cell
def get_demand_multipliers(poi: gpd.GeoDataFrame, grid: gpd.GeoDataFrame, radius: float) -> dict:
    result: dict = {}
    poi_copy = poi.copy()
    poi_copy['geometry'] = poi_copy.buffer(radius)
    poi_copy['buffer_area'] = poi_copy.area
    poi_len = len(poi_copy)

    for poi_index, poi_item in poi_copy.iterrows():
        poi_buffer = poi_item.geometry
        category = poi_item['Category']
        candidates = gpd.sjoin(grid, gpd.GeoDataFrame(geometry=[poi_buffer], crs=grid.crs), how="inner", predicate="intersects")

        if candidates.empty:
            continue

        for grid_index, grid_row in candidates.iterrows():
            intersection_area = grid_row.geometry.intersection(poi_buffer).area
            if intersection_area <= 0:
                continue

            coverage = round((intersection_area / poi_item.buffer_area) * 0.5, 1)
            if coverage <= 0:
                continue

            grid_key = str(grid_index)
            if grid_key not in result:
                result[grid_key] = {}

            result[grid_key][category] = result[grid_key].get(category, 0) + coverage

        if (poi_index + 1) % 10000 == 0:
            print(f'Computed: {poi_index + 1}/{poi_len}')

    print(f'Finished: {poi_len}/{poi_len}')
    return result

grids = gpd.read_file(PATH, layer='reviewed-grid', columns=[])
multipliers = get_demand_multipliers(pois, grids, 50)
with open('../data/demand/multiplier.json', 'w', encoding='utf-8') as f:
    json.dump(multipliers, f, ensure_ascii=False)

Computed: 10000/141415
Computed: 20000/141415
Computed: 30000/141415
Computed: 40000/141415
Computed: 50000/141415
Computed: 60000/141415
Computed: 70000/141415
Computed: 80000/141415
Computed: 90000/141415
Computed: 100000/141415
Computed: 110000/141415
Computed: 120000/141415
Computed: 130000/141415
Computed: 140000/141415
Finished: 141415/141415


In [3]:
demand_multiplier_df = pd.read_json('../data/demand/multiplier.json').sort_index().T.sort_index().apply(lambda x: round(x, 1)).fillna(0)
display(demand_multiplier_df.iloc[:10, :6])
print(f'{len(demand_multiplier_df)} rows x {len(demand_multiplier_df.columns)} cols')

grids = gpd.read_file(PATH, layer='reviewed-grid')
grids['Multiplier'] = demand_multiplier_df.sum(axis=1, numeric_only=True).round(1)
grids.sort_values(by='Index').reset_index(drop=True).to_file(PATH, driver='GPKG', layer='demand-multiplier')

,批發,郵政,零售,餐飲
0,0.0,0.0,0.0,0.1
1,0.0,0.0,0.0,4.1
2,0.0,0.0,0.0,0.2
3,0.2,0.0,0.4,1.6
4,0.0,0.0,0.2,0.0
5,0.1,0.0,0.1,0.5
6,0.9,0.0,1.5,0.8
7,1.3,0.0,0.6,1.7
8,0.2,0.0,0.9,1.1
9,0.3,0.0,2.0,2.7


2565 rows x 4 cols


In [4]:
def get_cargo_lots() -> gpd.GeoDataFrame:
    cargo_lots = gpd.read_file('../data/supply/lots.gpkg', layer='cargo', columns=[])
    cargo_lots.geometry = cargo_lots.centroid
    return cargo_lots

cargos = get_cargo_lots()
display(cargos.head(10))
print(f'{len(cargos)} rows x {len(cargos.columns)} cols')

,geometry
0,POINT (300839.864 2771015.699)
1,POINT (301228.538 2771668.416)
2,POINT (305056.347 2774614.496)
3,POINT (308302.088 2774510.741)
4,POINT (309026.185 2773522.547)
5,POINT (307608.989 2775109.724)
6,POINT (306971.018 2772408.566)
7,POINT (306547.826 2764624.55)
8,POINT (306831.697 2763903.909)
9,POINT (303921.617 2767503.021)


2070 rows x 1 cols


In [5]:
# Calculate the supply multiplier for each category in each grid cell
def get_supply_multipliers(lot: gpd.GeoDataFrame, grid: gpd.GeoDataFrame, car_per_hour: float, radius: float) -> dict:
    result = {}
    lot_copy = lot.copy()
    lot_copy.geometry = lot.buffer(radius)
    lot_copy['buffer_area'] = lot_copy.area

    for lot_index, lot_item in lot_copy.iterrows():
        lot_buffer = lot_item.geometry
        candidates = gpd.sjoin(grid, gpd.GeoDataFrame(geometry=[lot_buffer], crs=grid.crs), how="inner", predicate="intersects")

        if candidates.empty:
            continue

        for grid_index, grid_row in candidates.iterrows():
            intersection_area = grid_row.geometry.intersection(lot_buffer).area
            if intersection_area <= 0:
                continue

            supply = round((intersection_area / lot_item.buffer_area) * 8 * car_per_hour, 1)    # 8hr
            if supply <= 0:
                continue

            grid_key = str(grid_index)
            if grid_key not in result:
                result[grid_key] = supply
            else:
                result[grid_key] = round(result[grid_key] + supply, 1)

    return result

grids = gpd.read_file(PATH, layer='reviewed-grid', columns=[])
multipliers = get_supply_multipliers(cargos, grids, 1, 50)
with open('../data/supply/multiplier.json', 'w', encoding='utf-8') as f:
    json.dump(multipliers, f, ensure_ascii=False)

In [6]:
with open('../data/supply/multiplier.json', 'r', encoding='utf-8') as f:
    supply_multiplier = json.load(f)

supply_series = pd.Series(supply_multiplier, name='Multiplier', dtype=float)
supply_series.index = supply_series.index.astype(int)
display(supply_series.head(10))
print(f'{len(supply_series)} rows')

grids = gpd.read_file(PATH, layer='reviewed-grid')
grids['Multiplier'] = supply_series
grids.sort_values(by='Index').reset_index(drop=True).to_file(PATH, driver='GPKG', layer='supply-multiplier')

252      2.1
295     65.1
338      3.3
339      3.0
396     51.8
397     18.1
1370    26.9
1426    16.1
2166     6.4
2167    17.5
Name: Multiplier, dtype: float64

1400 rows


In [8]:
def get_grid_stats():
    grids = gpd.read_file(PATH, layer='reviewed-grid')
    demand = pd.read_json('../data/demand/multiplier.json').sort_index().T.sort_index().apply(lambda x: round(x, 1))

    with open('../data/supply/multiplier.json', 'r', encoding='utf-8') as f:
        supply_multiplier = json.load(f)

    supply = pd.Series(supply_multiplier, name='Multiplier', dtype=float)
    supply.index = supply.index.astype(float)

    grids = grids.join(demand)
    grids['供給'] = supply
    grids = grids.fillna(0)
    grids = gpd.GeoDataFrame(grids[['Index', '批發', '零售', '郵政', '餐飲', '供給', 'geometry']]).to_crs(4326)
    display(grids)
    return grids

get_grid_stats().to_file('../data/location/grid.geojson', index=False)

,Index,批發,零售,郵政,餐飲,供給,geometry
0,1,0.0,0.0,0.0,0.1,0.0,"POLYGON ((121.46231 25.11961, 121.4643 25.1196..."
1,2,0.0,0.0,0.0,4.1,0.0,"POLYGON ((121.46231 25.11781, 121.46429 25.117..."
2,3,0.0,0.0,0.0,0.2,0.0,"POLYGON ((121.46435 25.13224, 121.46633 25.132..."
3,4,0.2,0.4,0.0,1.6,0.0,"POLYGON ((121.46434 25.13044, 121.46632 25.130..."
4,5,0.0,0.2,0.0,0.0,0.0,"POLYGON ((121.46433 25.12863, 121.46632 25.128..."
...,...,...,...,...,...,...,...
2738,2739,1.0,2.5,0.0,4.7,2.1,"POLYGON ((121.62058 25.04139, 121.62256 25.041..."
2739,2740,4.0,6.1,0.0,8.7,13.9,"POLYGON ((121.62057 25.03959, 121.62255 25.039..."
2740,2741,1.4,1.4,0.0,0.4,0.0,"POLYGON ((121.62056 25.03778, 121.62254 25.037..."
2741,2742,0.7,0.9,0.0,0.4,0.0,"POLYGON ((121.62255 25.03958, 121.62453 25.039..."
